# Coordinate transformations and Error Propagation

The idea is to explore different options to propagate errors from observables ($\alpha$, $\delta$, $\varpi$, $\mu_{\alpha*}$, $\mu_\delta$ and $V_r$) to Cartesian Heliocentric Velocity. In between, we shall see also transformations to intermediate coordinate systems (basically Galactic spherical coordinates).

We shall see three ways:
- Astropy
- PyGaia
- GalPy _(soon)_
- Python Code  

__(¡¡WATCH OUT!! Parallax error -> the Jacobian is asuming that distance = 1/plx)__

For each one, we will average a thousand executions using _timeit_ package and obtain an estimated time cost.

In [1]:
import timeit
import numpy as np

In [26]:
""" Test star coordinates & errors """
    #J2000
ra=266.40506655 #right ascention in degrees
dec=-28.93616241 #declination in degrees
plx=4 #parallax in mas
pmra=2 #proper motion in alpha* in mas/yr
pmdec=3 #proper motion in delta in mas/yr
vr=0 #radial velocity in km/s

e_ra=0.1 #error in RA in mas
e_dec=0.1 #error in DEC in mas
e_plx=0.3 #error in plx in mas
e_pmra=0.7 #error in PMRA in mas/yr
e_pmdec=0.7 #error in PMDEC in mas/yr
e_vr=0 #error in Vr in km/s


""" Correct values based on NED calculator (ned.ipac.caltech.edu)
l=0 degrees
b=0 degrees
d=250 pc (1/plx)
 """

' Correct values based on NED calculator (ned.ipac.caltech.edu) for coordinates and PyGaia for velocities\nl=0 degrees\nb=0 degrees\nd=250 pc (1/plx)\nu=0\nv=\nw=\n '

## 1) Astropy


In [50]:
from astropy import units as u
from astropy.coordinates import SkyCoord,Galactocentric
from astropy.coordinates import HeliocentricTrueEcliptic,Galactic,LSR,HCRS

In [46]:
star=SkyCoord(ra=ra*u.degree, dec=dec*u.degree,
                distance=(plx*u.mas).to(u.pc, u.parallax()),
                pm_ra_cosdec=pmra*u.mas/u.yr,
                pm_dec=pmdec*u.mas/u.yr,
                radial_velocity=vr*u.km/u.s)

In [47]:
star

<SkyCoord (ICRS): (ra, dec, distance) in (deg, deg, pc)
    (266.40506655, -28.93616241, 250.)
 (pm_ra_cosdec, pm_dec, radial_velocity) in (mas / yr, mas / yr, km / s)
    (2., 3., 0.)>

In [58]:
""" Part I: change of coordinates """
#A: ICRS to Galactic
star_GAL=star.transform_to(Galactic)
tAstro=timeit.timeit(stmt='star.transform_to(Galactic)',globals=globals(),number=1000)/1000

In [60]:
print('Astropy\n\tStar at ({} deg:{} deg) in ICRS ->\
({}:{}) in Gal.Coord.\n\nTime: {} seconds'.format(ra,de,star_GAL.l,star_GAL.b,tAstro))

Astropy
	Star at (266.40506655 deg:-28.93616241 deg) in ICRS ->(4.879020186417789e-05 deg:-5.046269002433875e-05 deg) in Gal.Coord.

Time: 0.10792035559428405 seconds


In [61]:
#B: ICRS to Galactocentric (http://docs.astropy.org/en/stable/generated/examples/coordinates/plot_galactocentric-frame.html)
star_cart = star.transform_to(Galactocentric)
tAstro=timeit.timeit(stmt='star.transform_to(Galactocentric)',globals=globals(),number=1000)/1000

In [62]:
print(star_cart.x,star_cart.y,star_cart.z)
print(star_cart.v_x,star_cart.v_y,star_cart.v_z)
print('Time: {}'.format(tAstro))

-8049.957406537863 pc -1.9663881033693542e-05 pc 26.18688463399134 pc
11.09944533596824 km / s 236.50959376672404 km / s 7.0793609528452714 km / s
Time: 0.07135713972160784


In [ ]:
""" Part II: error propagation """
#As far as I know, not available -in a suitable way- in version 2.02 

## 2) PyGaia


In [3]:
import pygaia.astrometry.vectorastrometry as vecast
from pygaia.astrometry.coordinates import CoordinateTransformation
from pygaia.astrometry.coordinates import Transformations

In [15]:
""" Part I: change of coordinates """

#A: ICRS to GAL
    #define the transformation
ICRS2GAL=CoordinateTransformation(Transformations.ICRS2GAL)

    #use the methods to transform: first the position
l,b=ICRS2GAL.transformSkyCoordinates(np.deg2rad(ra),np.deg2rad(dec))
tGaiaCoord=timeit.timeit(stmt='ICRS2GAL.transformSkyCoordinates(np.deg2rad(ra),np.deg2rad(dec))',
                    globals=globals(),number=1000)/1000

    #then the proper motions
mul,mub=ICRS2GAL.transformProperMotions(np.deg2rad(ra),np.deg2rad(dec),pmra,pmdec)
tGaiaPM=timeit.timeit(stmt='ICRS2GAL.transformProperMotions(np.deg2rad(ra),np.deg2rad(dec),pmra,pmdec)',
                    globals=globals(),number=1000)/1000

In [17]:
print('PyGaia\n\tStar at ({} deg:{} deg) in ICRS -> ({} deg:{} deg) in Gal.Coord.\n\nTime: {} seconds'.format(
    ra,dec,np.rad2deg(l),np.rad2deg(b),tGaiaCoord))

PyGaia
	Star at (266.40506655 deg:-28.93616241 deg) in ICRS -> (4.257483637742752e-05 deg:-4.7577247564556497e-05 deg) in Gal.Coord.

Time: 4.148723054834136e-05 seconds


In [18]:
print('PyGaia\n\tStar at ({} mas/yr:{} mas/yr) in ICRS -> ({} mas/yr:{} mas/yr) in Gal.Coord.\n\nTime: {} seconds'.format(
    pmra,pmdec,mul,mub,tGaiaPM))

PyGaia
	Star at (2 mas/yr:3 mas/yr) in ICRS -> (3.6026749209392523 mas/yr:-0.1439910206761177 mas/yr) in Gal.Coord.

Time: 0.00011272465155843747 seconds


In [20]:
#B: GAL to Helio-cartesian
    #to change to cartesian, we use the module 'vecast'
x,y,z,u,v,w=vecast.astrometryToPhaseSpace(l,b,plx,mul,mub,vr)
tGaia=timeit.timeit(stmt='vecast.astrometryToPhaseSpace(l,b,plx,mul,mub,vr)',
                    globals=globals(),number=1000)/1000

In [21]:
print('PyGaia\n\tStar at ({} deg:{} deg:{} mas) in GAL -> ({} pc:{} pc:{} pc) in Heliocentric.Coord.\n\nTime:\
{} seconds'.format(
    np.rad2deg(l),np.rad2deg(b),plx,x,y,z,tGaia))

PyGaia
	Star at (4.257483637742752e-05 deg:-4.7577247564556497e-05 deg:4 mas) in GAL -> (249.9999999998448 pc:0.00018576776832091022 pc:-0.0002075949047594672 pc) in Heliocentric.Coord.

Time:4.8843137037806626e-05 seconds


In [27]:
#A+B:ICRS to Heliocentric Cartesian
    #full transformation in one function
def pygaiachange(ra,dec,plx,pmra,pmdec,vr):
    """ From observables in ICRS (angles in degrees, plx in mas, proper motion in mas/yr, los velocity in km/s)
    returns X,Y,Z (in pc) and U,V,W (in km/s)."""
    import pygaia.astrometry.vectorastrometry as vecast
    from pygaia.astrometry.coordinates import CoordinateTransformation
    from pygaia.astrometry.coordinates import Transformations   
    ICRS2GAL=CoordinateTransformation(Transformations.ICRS2GAL)
    #GAL2ICRS=CoordinateTransformation(Transformations.GAL2ICRS)

    l,b=ICRS2GAL.transformSkyCoordinates(np.deg2rad(ra),np.deg2rad(dec))
    mul,mub=ICRS2GAL.transformProperMotions(np.deg2rad(ra),np.deg2rad(dec),pmra,pmdec)
    
    return vecast.astrometryToPhaseSpace(l,b,plx,mul,mub,vr)

x,y,z,u,v,w=pygaiachange(ra,dec,plx,pmra,pmdec,vr)
tGaia=timeit.timeit(stmt='pygaiachange(ra,dec,plx,pmra,pmdec,vr)',
                    globals=globals(),number=1000)/1000

In [24]:
print('PyGaia\n\tStar at ({} deg:{} deg:{} mas) in ICRS -> ({} pc:{} pc:{} pc) in Heliocentric.Coord.\n\nTime:\
{} seconds'.format(
    ra,dec,plx,x,y,z,tGaia))

PyGaia
	Star at (266.40506655 deg:-28.93616241 deg:4 mas) in ICRS -> (249.9999999998448 pc:0.00018576776832091022 pc:-0.0002075949047594672 pc) in Heliocentric.Coord.

Time:0.00020382055854838653 seconds


In [25]:
print('PyGaia\n\tStar at ({} mas/yr:{} mas/yr:{} km/s) in ICRS -> ({} kms/s:{} km/s:{} km/s) in Heliocentric.Coord.\n\nTime:\
{} seconds'.format(
    pmra,pmdec,vr,u,v,w,tGaia))

PyGaia
	Star at (2 mas/yr:3 mas/yr:0 km/s) in ICRS -> (-3.3143126398119365e-06 kms/s:4.269593513104933 km/s:-0.1706462951322301 km/s) in Heliocentric.Coord.

Time:0.00020382055854838653 seconds


In [ ]:
""" Part II: error propagation (only rotations)"""

"""
Version 1.2 (December 2016)
++++++++++++++++++++
- Add method to CoordinateTransformation for the transformation of the full (5x5) covariance matrix of
  the astrometric parameters.

- Add keyword to astrometric errors prediction functions that allows to specify an extended mission
  lifetime.
  
  
+  def transformCovarianceMatrix(self, phi, theta, covmat):
+      
+      Transform the astrometric covariance matrix to its representation in the new coordinate system.
+
+      Parameters
+      ----------
+
+      phi       - The longitude-like angle of the position of the source (radians).
+      theta     - The latitude-like angle of the position of the source (radians).
+      covmat    - Covariance matrix (5x5) of the astrometric parameters.
+
+      Returns
+      -------
+
+      covmat_rot - Covariance matrix in its representation in the new coordinate system.
+      
+
+      c, s = self._getJacobian(phi,theta)
+      jacobian = identity(5)
+      jacobian[0][0]=c
+      jacobian[1][1]=c
+      jacobian[3][3]=c
+      jacobian[4][4]=c
+      jacobian[0][1]=s
+      jacobian[1][0]=-s
+      jacobian[3][4]=s
+      jacobian[4][3]=-s
+
+      return dot( dot(jacobian, covmat), jacobian.transpose() )
+
   def _getJacobian(self, phi, theta):
     
     Calculates the Jacobian for the transformation of the position errors and proper motion errors
     between coordinate systems. This Jacobian is also the rotation matrix for the transformation of
     proper motions. See section 1.5.3 of the Hipparcos Explanatory Volume 1 (equation 1.5.20).
 
     Parameters
     ----------
 
     phi       - The longitude-like angle of the position of the source (radians).
     theta     - The latitude-like angle of the position of the source (radians).
 
     Returns
     -------
 
     jacobian - The Jacobian matrix corresponding to (phi, theta) and the currently desired coordinate
                system transformation.
     
 
     p, q, r = normalTriad(phi, theta)
 
     # zRot = z-axis of new coordinate system expressed in terms of old system
     zRot = self.rotationMatrix[2,:]
     zRotAll = zRot
     if (p.ndim == 2):
       for i in range(p.shape[1]-1):
         zRotAll = vstack((zRotAll,zRot))
     pRot = cross(zRotAll, transpose(r))
     if (p.ndim == 2):
       normPRot = sqrt(diag(dot(pRot,transpose(pRot))))
       for i in range(pRot.shape[0]):
         pRot[i,:] = pRot[i,:]/normPRot[i]
     else:
       pRot = pRot/norm(pRot)
 
     if (p.ndim == 2):
       return diag(dot(pRot,p)), diag(dot(pRot,q))
     else:
return dot(pRot,p), dot(pRot,q)
"""

#Since the transformation is nested inside the 'CoordinateTransformation' method, it is only available for
#changes of coordinates defined in 'Transfromations' object. That is: ICRS<->GAL<->Ecliptic

In [40]:
ICRS2GAL=CoordinateTransformation(Transformations.ICRS2GAL)
help(ICRS2GAL.transformCovarianceMatrix)

Help on method transformCovarianceMatrix in module pygaia.astrometry.coordinates:

transformCovarianceMatrix(phi, theta, covmat) method of pygaia.astrometry.coordinates.CoordinateTransformation instance
    Transform the astrometric covariance matrix to its representation in the new coordinate system.
    
    Parameters
    ----------
    
    phi       - The longitude-like angle of the position of the source (radians).
    theta     - The latitude-like angle of the position of the source (radians).
    covmat    - Covariance matrix (5x5) of the astrometric parameters.
    
    Returns
    -------
    
    covmat_rot - Covariance matrix in its representation in the new coordinate system.



In [37]:
GALcovMatrix=ICRS2GAL.transformCovarianceMatrix(ra,dec,np.diag([e_ra,e_dec,e_plx,e_pmra,e_pmdec]))

In [38]:
print(GALcovMatrix)

[[ 0.1  0.   0.   0.   0. ]
 [ 0.   0.1  0.   0.   0. ]
 [ 0.   0.   0.3  0.   0. ]
 [ 0.   0.   0.   0.7  0. ]
 [ 0.   0.   0.   0.   0.7]]


## 3) Python Code

In [65]:
from Jacobian import *

In [72]:
""" Part I: change of coordinates """
#A: ICRS to Galactic
    #position
l,b=radec2lb(np.deg2rad(ra),np.deg2rad(de))
tPythonCoord=timeit.timeit(stmt='radec2lb(ra,de)',
                    globals=globals(),number=1000)/1000
    #proper motions
mul,mub=pmradec2lb(np.deg2rad(ra),np.deg2rad(de),l,b,pmra,pmdec)
tPythonPM=timeit.timeit(stmt='pmradec2lb(np.deg2rad(ra),np.deg2rad(de),l,b,pmra,pmdec)',
                    globals=globals(),number=1000)/1000

In [73]:
print('Python Code\n\tStar at ({} deg:{} deg) in ICRS -> ({} deg:{} deg) in Gal.Coord.\n\nTime: {} seconds'.format(
    ra,de,np.rad2deg(l),np.rad2deg(b),tPythonCoord))

print('Python\n\tStar at ({} mas/yr:{} mas/yr) in ICRS -> ({} kms/s:{} km/s) in Gal.Coord.\n\nTime:\
{} seconds'.format(
    pmra,pmdec,mul,mub,tPythonPM))

Python Code
	Star at (266.40506655 deg:-28.93616241 deg) in ICRS -> (0.004927143400317665 deg:-0.00031906842240168637 deg) in Gal.Coord.

Time: 3.5893787766781315e-05 seconds
Python
	Star at (2 mas/yr:3 mas/yr) in ICRS -> (3.602669678523336 kms/s:-0.143998792137229 km/s) in Gal.Coord.

Time:2.033684554271531e-05 seconds


In [80]:
""" Part II: error propagation """
#From ra,dec,plx,pmra,pmdec,vr to l,b,plx,U,V,W
J6=Jacob([ra,de,plx,pmra,pmdec,0])
J4=Jacob4([ra,de,plx,pmra,pmdec,0])

In [81]:
print(J6)
print(J4)

[[ 2.21069641e-06  4.13805520e-06  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [-3.62145761e-06  2.52605022e-06  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [-1.00568044e-05 -1.72367036e-05  9.20282436e-05 -5.87330660e-05
  -8.35491151e-05  9.99999996e-01]
 [-4.01173914e-07 -1.47789326e-09 -1.06739683e+00  6.17478391e-01
   1.01154546e+00  8.59948749e-05]
 [-1.00151765e-05  2.05442570e-10  4.26638766e-02 -1.01154546e+00
   6.17478394e-01 -5.56879451e-06]]
[[ 1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 9.20282436e-05 -5.87330660e-05 -8.35491151e-05  9.99999996e-01]
 [-1.06739683e+00  6.17478391e-01  1.01154546e+00  8.59948749e-05]
 [ 4.26638766e-02 -1.01154546e+00  6.17478394e-01 -5.56879451e-06]]


In [82]:
tJ6=timeit.timeit(stmt='Jacob([ra,de,plx,pmra,pmdec,0])',
                    globals=globals(),number=1000)/1000
tJ4=timeit.timeit(stmt='Jacob4([ra,de,plx,pmra,pmdec,0])',
                    globals=globals(),number=1000)/1000

In [83]:
print('Time [s]: ',tJ6,'/',tJ4)

Time [s]:  0.00027954947226589867 / 9.28001142156063e-05


In [88]:
print('Time to process Error Propagation: ',tJ4)
print('\nOriginal Covariance Matrix: ')
cov=np.diag([e_plx,e_pmra,e_pmdec,e_vr])**2
print(cov)
print('\nPropagated Covariance Matrix: ')
new_cov=J4@cov@J4.T
print(np.round(new_cov,2))

Time to process Error Propagation:  9.28001142156063e-05

Original Covariance Matrix: 
[[0.09 0.   0.   0.  ]
 [0.   0.49 0.   0.  ]
 [0.   0.   0.49 0.  ]
 [0.   0.   0.   0.  ]]

Propagated Covariance Matrix: 
[[ 0.09  0.   -0.1   0.  ]
 [ 0.    0.   -0.    0.  ]
 [-0.1  -0.    0.79 -0.  ]
 [ 0.    0.   -0.    0.69]]
